In [1]:
!pip install tensorboardX
!pip install ml_collections
!pip install monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.2 MB/s eta 0:00:00


In [2]:
!git clone --branch training https://github.com/lesnikowka/TRUNet.git

!pip install torchio
!pip install tensorflow===2.15.0

Cloning into 'TRUNet'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 104 (delta 46), reused 54 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (104/104), 7.70 MiB | 16.22 MiB/s, done.
Resolving deltas: 100% (46/46), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
%autosave 60

import os
import sys
import random
import numpy as np
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv3D
from keras.layers import Conv3DTranspose
from keras.layers import MaxPool3D
from keras.layers import UpSampling3D
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import MaxPool2D
from keras.layers import UpSampling2D
from keras.layers import Concatenate
from keras.layers import Flatten
from google.colab import drive
import cv2
import torchio as tio
import torch
import gc
import keras.backend as K
from datetime import datetime
from google.colab import files
import zipfile
import random
from skimage.transform import resize

Autosaving every 60 seconds


In [4]:
import os
print(os.getcwd())
import sys
sys.path.append('/content/TRUNet')

/content


In [5]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
archive = "2048cropped.zip"
!cp -r drive/MyDrive/arrays/$archive /content/sample_data/$archive
zipfile.ZipFile("/content/sample_data/" + archive, 'r').extractall("/content/sample_data")

In [7]:
NUM_CLASSES = 8
IMAGE_SIZE = 96

In [12]:
import argparse
import ml_collections
import os
from glob import glob
import torch
import numpy as np
from monai.losses import DiceCELoss
from monai.metrics import DiceMetric
from TRUNet.TRUNet_network.model.ViT import VisionTransformer3d as TransUNet3d
from datetime import datetime
from TRUNet_network.trunet_train import trainer
from torchvision import transforms
from TRUNet_network.augmentations import RandomGenerator3d_zoom, Reshape3d_zoom

now = datetime.now()

def TransUNet_configs(img_size):
    configs_trunet = ml_collections.ConfigDict()

    configs_trunet.resnet = ml_collections.ConfigDict()
    configs_trunet.resnet.num_layers = (3, 4, 9)
    configs_trunet.resnet.width_factor = 1
    configs_trunet.transformer_mlp_dim = 3072
    configs_trunet.transformer_num_heads = 12
    configs_trunet.transformer_num_layers = 12
    configs_trunet.transformer_attention_dropout_rate = 0.0
    configs_trunet.transformer_dropout_rate = 0.1
    configs_trunet.classifier = 'seg'
    configs_trunet.decoder_channels = (256, 128, 64, 16)
    configs_trunet.n_classes = 7
    configs_trunet.n_skip = 3
    configs_trunet.skip_channels = [512, 256, 64, 16]
    configs_trunet.patches = ml_collections.ConfigDict()
    configs_trunet.patches.grid = None

    configs_trunet.hidden_size = 768
    configs_trunet.patches.size = 16

    configs_trunet.patch_size = configs_trunet.patches.size  # (results in 14 by 14 grid of patches for input size 224)

    configs_trunet.patches.grid = (
        int(img_size / configs_trunet.patches.size), int(img_size / configs_trunet.patches.size),
        int(img_size / configs_trunet.patches.size))
    configs_trunet.hybrid = True

    return configs_trunet


class fetch_dataset:
    def __init__(self, base_dir, crop=None, transform=None):
        self.transform = transform
        self.data_dir = base_dir
        sample_list = sorted(glob(os.path.join(base_dir, '*.npy')))
        self.sample_list = sample_list
        self.name = [i.split('/')[-1].split('.npy')[0] for i in sample_list]
        self.pt = [int(i.split('_')[0][2:]) for i in self.name]
        self.crop = crop
        if crop is None or crop == 'None':
            pass
        else:
            with open(crop) as file:
                lines = [line.rstrip() for line in file]
            self.crop_pt = [int(i.split(' ')[0]) for i in lines]
            self.crop_xmin = [float(i.split(' ')[1]) for i in lines]
            self.crop_xmax = [float(i.split(' ')[2]) for i in lines]
            self.crop_ymin = [float(i.split(' ')[3]) for i in lines]
            self.crop_ymax = [float(i.split(' ')[4]) for i in lines]
            self.crop_zmin = [float(i.split(' ')[5]) for i in lines]
            self.crop_zmax = [float(i.split(' ')[6]) for i in lines]

    def __len__(self):
        return len(self.sample_list)

    def __getitem__(self, idx):
        data_path = self.sample_list[idx]
        data = np.load(data_path)
        image, label = data['arr_0'], data['arr_1']

        if self.crop is None or self.crop == 'None':
            pass
        else:
            idx_crop = self.crop_pt.index(self.pt[idx])
            idx_crop = int(idx_crop)
            x, y, z = image.shape
            xmin = max([0, int(self.crop_xmin[idx_crop]) - 20])
            ymin = max([0, int(self.crop_ymin[idx_crop]) - 20])
            zmin = max([0, int(self.crop_zmin[idx_crop]) - 20])
            xmax = min([x, int(self.crop_xmax[idx_crop]) + 20])
            ymax = min([y, int(self.crop_ymax[idx_crop]) + 20])
            zmax = min([z, int(self.crop_zmax[idx_crop]) + 20])
            image = image[xmin:xmax, ymin:ymax, zmin:zmax]
            label = label[xmin:xmax, ymin:ymax, zmin:zmax]

        sample = {'image': image, 'label': label, 'case_name': self.sample_list[idx].split('/')[-1].split('.npz')[0]}
        if self.transform:
            sample = self.transform(sample)

        return sample


if __name__ == "__main__":

    args = ml_collections.ConfigDict()
    args.max_epochs = 2
    args.save_path = "save_path"
    args.root_path = "sample_dir"
    args.num_classes = NUM_CLASSES
    args.batch_size = 1
    args.seed = 42
    args.base_lr = 0.01

    # Transforms & Augmentations
    train_transforms = transforms.Compose(
        [RandomGenerator3d_zoom(output_size=(IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE))])
    val_transforms = transforms.Compose([Reshape3d_zoom(output_size=[IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE])])

    # Define model
    config_net = TransUNet_configs(IMAGE_SIZE)
    config_net.n_classes = args.num_classes
    model = TransUNet3d(config_net, img_size=IMAGE_SIZE, zero_head=False,
                        vis=False)
    config = {'ds_val': fetch_dataset(base_dir=os.path.join(args.root_path, 'images'), transform=val_transforms),
              'ds_train': fetch_dataset(base_dir=os.path.join(args.root_path, 'masks'), transform=train_transforms),
              'loss_function': DiceCELoss(include_background=False, to_onehot_y=True, softmax=True),
              'metric': DiceMetric(include_background=False, reduction="mean"),
              'optimizer': torch.optim.Adam(model.parameters(), args.base_lr),
              'save_interval': 50}

    trainer(args, config, model, args.save_path)

ValueError: num_samples should be a positive integer value, but got num_samples=0